In [1]:
# load packages
import os
import sys
import re
import numpy as np
import tensorflow as tf
from sklearn.datasets import load_boston
tf.__version__

'1.12.0'

In [2]:
# load data
boston = load_boston()
features = np.array(boston.data)
labels = np.array(boston.target)
features.shape, labels.shape

((506, 13), (506,))

In [3]:
labels = labels.reshape((len(labels), 1)) # reshape rank 1 array to have 1 dim
labels.shape

(506, 1)

In [4]:
features[0]

array([6.320e-03, 1.800e+01, 2.310e+00, 0.000e+00, 5.380e-01, 6.575e+00,
       6.520e+01, 4.090e+00, 1.000e+00, 2.960e+02, 1.530e+01, 3.969e+02,
       4.980e+00])

In [5]:
labels[0]

array([24.])

In [6]:
mean = np.mean(features, axis=0) # feature wise mean
std = np.std(features, axis=0) # feature wise standard deviation
# normalize feature now
features = (features - mean) / std
features.shape

(506, 13)

In [7]:
features[0]

array([-0.41978194,  0.28482986, -1.2879095 , -0.27259857, -0.14421743,
        0.41367189, -0.12001342,  0.1402136 , -0.98284286, -0.66660821,
       -1.45900038,  0.44105193, -1.0755623 ])

In [8]:
random_indices = np.random.rand(len(features)) < 0.80 # random indexes for splitting
# splti data
train_x = features[random_indices]
train_y = labels[random_indices]
val_x = features[~random_indices]
val_y = labels[~random_indices]

In [9]:
# set dimension
num_features = features.shape[1]
num_features

13

In [20]:
# create placeholders for input
x = tf.placeholder(dtype=tf.float32, shape=(None, num_features), name="feature_matrix") # num_samples x num_features
y = tf.placeholder(dtype=tf.float32, shape=(None, 1), name="target_vector") # num_sampels X 1

In [21]:
# define weight for the model
w = tf.Variable(tf.zeros(dtype=tf.float32, shape=(1, num_features)))

In [22]:
# define bais
b = tf.Variable(tf.zeros(dtype=tf.float32, shape=(1, num_features)))

In [23]:
# linear model
linear_model = tf.add(tf.multiply(w, x), b)

In [24]:
loss_function = tf.reduce_mean(tf.square(linear_model - y)) # define loss function

### using Inbuilt Adam Optimizer

In [25]:
trainer = tf.train.AdamOptimizer(learning_rate=0.01).minimize(loss_function) # trainer

In [26]:
# run the mode
with tf.Session() as sess:
    epochs = 10000
    print("[Training...]\n")
    sess.run(tf.global_variables_initializer()) # initialize variables
    # run the optimizer
    for i in range(epochs+1):
        sess.run(trainer, feed_dict={x:train_x, y:train_y}) # train and update weights and bais
        c = sess.run(loss_function, feed_dict={x:train_x, y:train_y}) # get the loss for current w and b
        if i % 1000 == 0:
            print("Step", i, ":", end="\t")
            print("Train Loss: ", c)
    print("Validation Loss: ", sess.run(loss_function, feed_dict={x:val_x, y:val_y}))
    print("Final Weight:", sess.run(w))
    print("Final Bias:", sess.run(b))
    print()

[Training...]

Step 0 :	Train Loss:  597.1853
Step 1000 :	Train Loss:  255.944
Step 2000 :	Train Loss:  116.56724
Step 3000 :	Train Loss:  75.38359
Step 4000 :	Train Loss:  69.538506
Step 5000 :	Train Loss:  69.31896
Step 6000 :	Train Loss:  69.3181
Step 7000 :	Train Loss:  69.3181
Step 8000 :	Train Loss:  69.31809
Step 9000 :	Train Loss:  69.3181
Step 10000 :	Train Loss:  69.318115
Validation Loss:  58.303604
Final Weight: [[-3.2571695  3.0029676 -4.342824   2.1187878 -3.7252488  6.3930073
  -3.1344957  2.1410244 -3.4117022 -4.2930226 -4.456309   3.0973756
  -6.856764 ]]
Final Bias: [[22.60375  22.745926 22.70757  22.613136 22.62807  22.625416 22.64883
  22.692284 22.579054 22.615358 22.473026 22.655428 22.565424]]



### Using Gradient From Scratch

In [15]:
grad_w, grad_b = tf.gradients(xs=[w, b], ys=loss_function) # compute gradient

In [16]:
# update weights and bias
new_w = w.assign(w - 0.01 * grad_w) # assign new value
new_b = b.assign(b - 0.01 * grad_b)

In [17]:
# run the mode
with tf.Session() as sess:
    epochs = 10000
    print("[Training...]\n")
    sess.run(tf.global_variables_initializer()) # initialize variables
    # run the optimizer for 1000 iterations
    for i in range(epochs+1):
        _, _, c = sess.run([new_w, new_b, loss_function], feed_dict={x:train_x, y:train_y})
        if i % 1000 == 0:
            print("Step", i, ":", end="\t")
            print("Train Loss: ", c)
    print("Validation Loss: ", sess.run(loss_function, feed_dict={x:val_x, y:val_y}))
    print("Final Weights:", sess.run(w))
    print("Final Bias:", sess.run(b))
    print()

[Training...]

Step 0 :	Train Loss:  597.7122
Step 1000 :	Train Loss:  93.70311
Step 2000 :	Train Loss:  70.45078
Step 3000 :	Train Loss:  69.371155
Step 4000 :	Train Loss:  69.32062
Step 5000 :	Train Loss:  69.318214
Step 6000 :	Train Loss:  69.31811
Step 7000 :	Train Loss:  69.3181
Step 8000 :	Train Loss:  69.3181
Step 9000 :	Train Loss:  69.31811
Step 10000 :	Train Loss:  69.31811
Validation Loss:  58.30311
Final Weights: [[-3.2571068  3.0028489 -4.3426533  2.1187181 -3.7251704  6.392851
  -3.134415   2.1409216 -3.4116397 -4.2928643 -4.456479   3.0972924
  -6.8566117]]
Final Bias: [[22.603153 22.745323 22.706968 22.612541 22.627474 22.624819 22.648232
  22.691685 22.578457 22.614761 22.472424 22.65483  22.564829]]

